In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)


In [2]:
train = pd.read_csv('data_set_ALL_AML_train.csv')


In [4]:
test = pd.read_csv('data_set_ALL_AML_independent.csv')


In [57]:
train.shape, test.shape


((7129, 78), (7129, 70))

In [71]:
def get_person_columns(df):
    return [c for c in df.columns if c.isdigit()]


In [72]:
target_orig = pd.read_csv('actual.csv')
target_orig.shape


(72, 2)

In [73]:
train_expressions_cols = get_person_columns(
    train)  # list(map(str,range(1,38)))
test_expressions_cols = get_person_columns(
    test)  # list(map(str,range(39, 63)))
# expressions_cols


In [75]:
# train_expressions_cols


In [77]:
# train['Gene Accession Number']


In [78]:
train_exp = pd.DataFrame(train[train_expressions_cols]).set_index(
    train['Gene Accession Number'])
test_exp = pd.DataFrame(test[test_expressions_cols]).set_index(
    test['Gene Accession Number'])


In [79]:
import plotly.graph_objects as go

# fig = go.Figure(data=[go.Histogram(x=exp.values.reshape(-1))])
# fig.show()


In [99]:
X_train = train_exp.rename(
    {c: f"person{c}" for c in train_exp.columns}, axis='columns').T
X_test = test_exp.rename(
    {c: f"person{c}" for c in test_exp.columns}, axis='columns').T
val_size = 8
X_val = X_train[-val_size:]
X_train = X_train[:-val_size]


In [100]:
X_train.shape, X_val.shape, X_test.shape


((30, 7129), (8, 7129), (34, 7129))

In [101]:
target = pd.Series(target_orig.cancer.values, index=[
                   f"person{n}" for n in target_orig.patient]).rename("cancer")
target_one_hot = (target == "ALL").astype(int)
target_one_hot.value_counts()
Y_train = target_one_hot.loc[X_train.index]
Y_val = target_one_hot.loc[X_val.index]
Y_test = target_one_hot.loc[X_test.index]


In [102]:
cors = X_train.apply(lambda x: x.corr(target_one_hot))


In [103]:
# persons.join(target)


In [104]:
cors = cors.sort_values()
cors


Gene Accession Number
M23197_at          -0.914267
M22960_at          -0.827920
U50136_rna1_at     -0.815395
D49950_at          -0.806200
X95735_at          -0.785799
                      ...   
M98343_at           0.644707
U37055_rna1_s_at    0.653387
U60319_at           0.659274
S82185_at           0.663764
L13266_s_at         0.666663
Length: 7129, dtype: float64

In [105]:
important_features = cors[cors.abs() > 0.7].index


In [106]:
# cors.iloc[:30]


In [107]:
# cors.iloc[-15:]


In [108]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [109]:
# persons


In [110]:
reg = LogisticRegression()
reg.fit(X_train, Y_train)
predicted = reg.predict(X_test)


In [111]:
predicted


array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0])

In [112]:
from sklearn.metrics import classification_report


In [113]:
print(classification_report(Y_test, predicted))


              precision    recall  f1-score   support

           0       0.82      0.64      0.72        14
           1       0.78      0.90      0.84        20

    accuracy                           0.79        34
   macro avg       0.80      0.77      0.78        34
weighted avg       0.80      0.79      0.79        34

